In [ ]:
#
from eye.gaze_track.datasets import UnityEyesDataset
from eye.gaze_track.augmentations import DataAugmentationImage, Preprocess
from eye.gaze_track.utils import draw_gaze

import matplotlib.pyplot as plt
import cv2

In [ ]:
load_name = "/content/drive/MyDrive/eye_w/weights/trans_2_3e4_att_256_1learnparam_noNorm_land_alt_MdataN-Step-Checkpoint_30_62160.ckpt"

proj_a = Gaze_Track_pl.load_from_checkpoint(load_name)

In [ ]:
#
dataset_val = UnityEyesDataset(img_dir = "./gazeset_2/imgs", val = True, transform = Preprocess(), grayscale = True, output_size = (96, 160), alt_land=True)
val_set = DataLoader(dataset_val, batch_size = 20)
#dd = DataAugmentationImage(1)

In [ ]:
#
xx = 0

for bb in val_set:
  xx = bb
  break

In [ ]:
w = 25
h = 25
fig = plt.figure(figsize=(24, 24))
columns = 5
rows = 4
for i in range(0, columns*rows):

    gaze, heatmaps, landmarks_out = proj_a(xx['img'][i][None])#.to('cuda'))
    xx['img'][i] = xx['img'][i][0]#.to('cpu')[0]

    gaze = gaze[0].detach().numpy()
    image = xx['img'][i].detach().numpy()
    landmarks = landmarks_out.detach().numpy()

    landmarks = proj_a.tranform_into_actual_coor(landmarks)
    landmarks = landmarks[0]

    eye_img = np.copy(image)
    eye_img = eye_img.transpose(1, 2, 0)
    eye_img = cv2.merge((eye_img,eye_img,eye_img))

    eye_img = cv2.normalize(eye_img, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    eye_img = eye_img.astype(np.uint8)

    landmarks[:, [1, 0]] = landmarks[:, [0, 1]]

    for (x, y) in landmarks[0:]:
        eye_img = cv2.circle(eye_img, (int(x), int(y)), 1, color=(0, 255, 0), thickness=3)
      
    eye_img = draw_gaze(eye_img, landmarks[-1], gaze, color=(255, 0, 0))

    gaze = xx['gaze'][i].detach().numpy()
    landmarks = np.copy(xx['landmarks'][i].detach().numpy())
    
    landmarks = proj_a.tranform_into_actual_coor(landmarks[None])
    landmarks = landmarks[0]

    landmarks[:, [1, 0]] = landmarks[:, [0, 1]]
    eye_img = draw_gaze(eye_img, landmarks[-1], gaze, color=(0, 255, 0))

    fig.add_subplot(rows, columns, i+1)
    plt.imshow(eye_img)

plt.show()

In [ ]:
add mpi visual testing!!!!